In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
FOLDERNAME = '빅데이터캡스톤/프젝/웹툰 크롤링된 이미지'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))

# Change dariectory to current folder
%cd /content/drive/MyDrive/$FOLDERNAME

/content/drive/MyDrive/빅데이터캡스톤/프젝/웹툰 크롤링된 이미지


In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.6/461.6 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless')

chrome_options.add_argument('--no-sandbox')

chrome_options.add_argument('--disable-dev-shm-usage')

chrome_options.add_argument(
            'user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36')

chrome_options.add_argument("disable-gpu")
chrome_options.add_argument("--lang=ko_KR")
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
# 정보들을 담을 리스트 정의하기
id_list = []
title_list = []
plot_list = []
tag_list = []
webtoon_url_list = []
img_url_list = []

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from time import sleep
import requests
import os
import requests
import re
import pandas as pd

In [ ]:
url = 'https://comic.naver.com/bestChallenge?sortType=view&page=1'
driver.get(url)

#User-Agent값으로 유저 정보 입력
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
res = requests.get(url, headers=headers)

# Create a BeautifulSoup object to parse the initial page content
soup = BeautifulSoup(res.text, 'html.parser')

titles = driver.find_elements(By.CLASS_NAME, "ChallengeListItem__item--X3Md7")
# Sleep for a short duration to allow the page to load
time.sleep(3)

for i in range(20, len(titles)): #앞에 크롤링한거 방지하기  위해 range에 숫자 explict
    driver.get(driver.current_url) #back하거나 새로고침되면 주소를 다시 가져와야함
    time.sleep(2)
    print('Current URL:', driver.current_url)
    print(len(titles))
    # Click on a specific element, e.g., the second element with class "ChallengeListItem__item--X3Md7"
    titles_tmp = driver.find_elements(By.CLASS_NAME, "ChallengeListItem__item--X3Md7")
    time.sleep(2)
    # basket_titles = titles_tmp
    # time.sleep(3)
    # print('basket_titles', basket_titles)
    print(titles_tmp, len(titles))
    titles_tmp[i].click()
    print(i, '/30   ' , driver.current_url)

    # Sleep for a short duration
    sleep(0.5)

    # Click on a specific title link (you can replace this XPath with the actual XPath)
    title_link = driver.find_element(By.XPATH, '//*[@id="content"]/ul/li/a')
    title_link.click()
    print(driver.current_url)
    webtoon_url_list.append(title_link)
    # Sleep for a short duration
    sleep(0.5)

    # Get the page source after clicking
    html = driver.page_source

    # Create a BeautifulSoup object to parse the clicked page content
    soup = BeautifulSoup(html, 'html.parser')

    # Get the title
    title_element = soup.select_one("#content div div h2")
    if(str(type(title_element)) == "<class 'NoneType'>"):
      print('title element is NoneType')
      continue;
    title = title_element.text
    title_list.append(title)
    print('Title:', title)
    print(driver.current_url)

    sleep(0.5)

    # Get the Plot
    plot_element = soup.select_one("#content div div div p")
    plot = plot_element.text
    plot_list.append(plot)
    print('Plot:', plot)
    print(driver.current_url)

    sleep(0.5)

    # Get the Tag
    div_element = soup.select_one('div.TagGroup__tag_group--uUJza')

    if div_element:
        # Extract text from all <a> tags within the selected div
        a_tags_text = [a.text for a in div_element.select('a')]
        tag_list.append(a_tags_text)
    else: "tag not Found"

    print('Tags', tag_list)
    print(driver.current_url)

    sleep(0.5)

    #회차 계산
    li_elements = driver.find_elements(By.XPATH, '//*[@id="content"]/div[3]/ul/li')
    if(len(li_elements) > 5): li_elements_len = 5
    else: li_elements_len = len(li_elements)
    print('li_elements', len(li_elements))
    sleep(0.5)

    #회차 다 돌아보기
    for j in range(li_elements_len):
      driver.get(driver.current_url) #back하거나 새로고침되면 주소를 다시 가져와야함
      print('Current URL:', driver.current_url)

      time.sleep(2)
      li_elements = driver.find_elements(By.XPATH, '//*[@id="content"]/div[3]/ul/li')
      basket_li_elements = li_elements
      print('li_elements 개수 :', len(li_elements))
      time.sleep(2)
      basket_li_elements[j].click()

      match = re.search(r'(?<=no=)\d+', driver.current_url) #url 뒤의 no= 파라미터로 회차 계산
      round = match.group()
      print(round, '회차')

      sleep(2)

      # Fetch the page content using requests
      page_content = requests.get(driver.current_url, headers=headers).text

      # Create a BeautifulSoup object with the page content
      soup = BeautifulSoup(page_content, 'html.parser')

      #이미지 순서대로 확인
      for k in range(10):
        element_id = f'content_image_{k}'  # Generate the element ID

        # Find the image element with the generated ID
        image_element = soup.find('img', {'id': element_id})

        sleep(0.5)
        # Check if the image element exists
        if image_element:
            image_src = image_element['src']
            img_url_list.append(image_src)
            print('imgsrc', image_src)

            sleep(0.5)

            #이미지 저장
            dir_now = os.getcwd()

            r = requests.get(image_src, headers=headers)

            with open(f'{dir_now}/{title}_{round}_{k}.jpg', "wb") as outfile:
                outfile.write(r.content)
            sleep(0.5)
        else:
            # Handle the case where the element does not exist
            print(f'Image with ID {element_id} not found')
            break;
      driver.back()
    driver.back()
    print(driver.current_url)  #https://comic.naver.com/artistTitle
    driver.back()
    print(driver.current_url) #https://comic.naver.com/bestChallenge?sortType=view&page=3
    print('backeddd')

Current URL: https://comic.naver.com/bestChallenge?sortType=view&page=1
30
[<selenium.webdriver.remote.webelement.WebElement (session="16dee5c87f86a197a30a1cf1c51ecfa6", element="1FCAEDB9B8F41016EF9090699F2EDEC3_element_181")>, <selenium.webdriver.remote.webelement.WebElement (session="16dee5c87f86a197a30a1cf1c51ecfa6", element="1FCAEDB9B8F41016EF9090699F2EDEC3_element_182")>, <selenium.webdriver.remote.webelement.WebElement (session="16dee5c87f86a197a30a1cf1c51ecfa6", element="1FCAEDB9B8F41016EF9090699F2EDEC3_element_183")>, <selenium.webdriver.remote.webelement.WebElement (session="16dee5c87f86a197a30a1cf1c51ecfa6", element="1FCAEDB9B8F41016EF9090699F2EDEC3_element_184")>, <selenium.webdriver.remote.webelement.WebElement (session="16dee5c87f86a197a30a1cf1c51ecfa6", element="1FCAEDB9B8F41016EF9090699F2EDEC3_element_185")>, <selenium.webdriver.remote.webelement.WebElement (session="16dee5c87f86a197a30a1cf1c51ecfa6", element="1FCAEDB9B8F41016EF9090699F2EDEC3_element_186")>, <selenium.we

KeyboardInterrupt: 

In [ ]:
print(title_list)
print(len(plot_list))
print(tag_list)
# 정보 저장
data = {
    'titles': title_list,
    'plots': plot_list,
    'tags': tag_list
}

df = pd.DataFrame(data)
# Specify the file path for the Excel file
excel_file_path = '/content/drive/MyDrive/빅데이터캡스톤/프젝/웹툰 정보_3p_3.xlsx'
df.to_excel(excel_file_path, index=False)

print(f"Excel file saved to: {excel_file_path}")

In [ ]:
# print(type(title_element))